# Import Libraries

In [1]:
#-*-coding:utf-8-*-

%load_ext autoreload
%autoreload 2

import sys
import os

import numpy as np
import pandas as pd

import torch

import time
import datetime

import warnings

__ROOT_PATH = os.path.abspath('../')
sys.path.append(__ROOT_PATH)

from vol_surface_manager.kap_constants import *
from vol_surface_manager.kap_parse_args import parse_args
from vol_surface_manager.kap_logger import acquire_logger
from vol_surface_manager.kap_data_loader import load_data_from_config, save_coef_data_from_config, save_selection_data_from_config
from vol_surface_manager.kap_main_data import daily_coefs_to_coef_data, daily_train_data_to_selection_data

from vol_surface_manager.kap_models import HyeonukLossModel1 as LossModel
from vol_surface_manager.kap_models import HyeonukCandidateModel1 as CandidateModel

#from vol_surface_manager.kap_models import HyeonukLossModel2 as LossModel
#from vol_surface_manager.kap_models import HyeonukCandidateModel2 as CandidateModel

#from vol_surface_manager.kap_models import HyeonukLossModel3 as LossModel
#from vol_surface_manager.kap_models import HyeonukCandidateModel3 as CandidateModel

#from vol_surface_manager.kap_models import HyeonukLossModel4 as LossModel
#from vol_surface_manager.kap_models import HyeonukCandidateModel4 as CandidateModel

from vol_surface_manager.kap_vol_surface import create_vol_surface_at_day

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Parse Arguments

In [2]:
#LOG = "DEBUG"
LOG = "INFO"
DATA_TYPE = "Kospi200"
#DATA_TYPE = "S&P500"
MODEL_BASE_DIR = "../models_svm"
RESULT_BASE_DIR = "../results/results_test"

# ignore by message
warnings.filterwarnings("ignore", message="overflow encountered in square")
warnings.filterwarnings("ignore", message="overflow encountered in multiply")
warnings.filterwarnings("ignore", message="invalid value encountered in double_scalars")
warnings.filterwarnings("ignore", message="divide by zero encountered in double_scalars")

# 0. Parsing arguments
sys.argv = ["kap_runme_vol_surface.py",
            "--LOG=%s" % LOG,
            "--DATA_TYPE=%s" % DATA_TYPE,
            "--MODEL_BASE_DIR=%s" % MODEL_BASE_DIR,
            "--RESULT_BASE_DIR=%s" % RESULT_BASE_DIR]
config = parse_args()
logger = acquire_logger(config)

# Acquire Market Data

In [3]:
# 2. Acquiring market data
market_data, raw_param_data, coef_data, all_days_init, coef_days, market_days = load_data_from_config(
    config, logger, mode='eval', acquire_market_data=False)
logger.info('Market data is loaded.')

    INFO - Market data is loaded.


# Load Models

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
try:
    assert(os.path.isdir(config.MODEL_BASE_DIR))
except Exception as e:
    logger.fatal('Invalid model directory: "%s."' % config.MODEL_BASE_DIR)
    raise e

# 2. Loading loss models
alpha_loss_model, rho_loss_model, nu_loss_model = None, None, None
LOSS_MODEL_DIRNAME = 'loss_models'
loss_model_base_dir = os.path.join(config.MODEL_BASE_DIR, LOSS_MODEL_DIRNAME)
loss_model_base_dir_2 = os.path.join(loss_model_base_dir, config.DATA_TYPE)
for parameter in ['Alpha', 'Rho', 'Nu']:
    loss_model_save_dir = os.path.join(loss_model_base_dir_2, parameter)
    loss_model_name = 'model_best'
    try:
        loss_model = LossModel(device, parameter=parameter)
        loss_model.load(loss_model_save_dir, loss_model_name)
        loss_model.eval()
        logger.info('%s loss model is loaded.' % parameter)
    except:
        loss_model = None
        logger.warning('No %s loss model exists.' % parameter)
    if parameter == 'Alpha':
        alpha_loss_model = loss_model
    elif parameter == 'Rho':
        rho_loss_model = loss_model
    else:
        assert(parameter == 'Nu')
        nu_loss_model = loss_model

# 3. Loading candidate models
alpha_candidate_model, rho_candidate_model, nu_candidate_model = None, None, None
CANDIDATE_MODEL_DIRNAME = 'candidate_models'
candidate_model_base_dir = os.path.join(config.MODEL_BASE_DIR, CANDIDATE_MODEL_DIRNAME)
candidate_model_base_dir_2 = os.path.join(candidate_model_base_dir, config.DATA_TYPE)
for parameter in ['Alpha', 'Rho', 'Nu']:
    candidate_model_save_dir = os.path.join(candidate_model_base_dir_2, parameter)
    candidate_model_name = 'model_best'
    try:
        candidate_model = CandidateModel(device, parameter=parameter)
        candidate_model.load(candidate_model_save_dir, candidate_model_name)
        candidate_model.eval()
        logger.info('%s candidate model is loaded.' % parameter)
    except:
        candidate_model = None
        logger.warning('No %s candidate model exists.' % parameter)
    if parameter == 'Alpha':
        alpha_candidate_model = candidate_model
    elif parameter == 'Rho':
        rho_candidate_model = candidate_model
    else:
        assert (parameter == 'Nu')
        nu_candidate_model = candidate_model

    INFO - Alpha loss model is loaded.
    INFO - Rho loss model is loaded.
    INFO - Nu loss model is loaded.
    INFO - Alpha candidate model is loaded.
    INFO - Rho candidate model is loaded.
    INFO - Nu candidate model is loaded.


In [8]:
alpha_loss_model = None
rho_loss_model = None
nu_loss_model = None

#alpha_candidate_model = None
#rho_candidate_model = None
#nu_candidate_model = None

# Create Vol Surface

In [9]:
all_days = []
for day_index in range(1, len(all_days_init)):
    yesterday = all_days_init[day_index-1]
    day = all_days_init[day_index]
    if not (yesterday in coef_days and day in coef_days):
        continue
    all_days.append(day)

indices = np.arange(len(all_days))
train_boundary = sum(day < datetime.datetime.strptime('20190101', '%Y%m%d') for day in all_days)
train_indices = indices[:train_boundary]
test_indices = indices[train_boundary:]
vol_dates = [all_days[index].strftime('%Y%m%d') for index in test_indices]
logger.info('Training dates are acquired.')

print(vol_dates)

    INFO - Training dates are acquired.


['20190101', '20190102', '20190103', '20190104', '20190107', '20190108', '20190109', '20190110', '20190111', '20190114', '20190115', '20190116', '20190117', '20190118', '20190121', '20190122', '20190123', '20190124', '20190125', '20190128', '20190129', '20190130', '20190131', '20190201', '20190204', '20190205', '20190206', '20190207', '20190208', '20190211', '20190212', '20190213', '20190214', '20190215', '20190218', '20190219', '20190220', '20190221', '20190222', '20190225', '20190226', '20190227', '20190228', '20190301', '20190304', '20190305', '20190306', '20190307', '20190308', '20190311', '20190312', '20190313', '20190314', '20190315', '20190318', '20190319', '20190320', '20190321', '20190322', '20190325', '20190326', '20190327', '20190328', '20190329', '20190401', '20190402', '20190403', '20190404', '20190405', '20190408', '20190409', '20190410', '20190411', '20190412', '20190415', '20190416', '20190417', '20190418', '20190419', '20190422', '20190423', '20190424', '20190425', '20

In [10]:
#pred_coef_data.to_parquet('temp_pred_coef_data.parquet', engine='pyarrow')
#pred_alpha_selection_data.to_parquet('temp_pred_alpha_selection_data.parquet', engine='pyarrow')
#pred_rho_selection_data.to_parquet('temp_pred_rho_selection_data.parquet', engine='pyarrow')
#pred_nu_selection_data.to_parquet('temp_pred_nu_selection_data.parquet', engine='pyarrow')
#pred_coef_data = pd.read_parquet('temp_pred_coef_data.parquet', engine='pyarrow')
#pred_alpha_selection_data = pd.read_parquet('temp_pred_alpha_selection_data.parquet', engine='pyarrow')
#pred_rho_selection_data = pd.read_parquet('temp_pred_rho_selection_data.parquet', engine='pyarrow')
#pred_nu_selection_data = pd.read_parquet('temp_pred_nu_selection_data.parquet', engine='pyarrow')

In [11]:
pred_coef_data = None
pred_alpha_selection_data = None
pred_rho_selection_data = None
pred_nu_selection_data = None

start_time = time.time()

for idate, vol_date in enumerate(vol_dates):
    #if idate < 9-1:
    #    continue
    # 1. Select date to create vol surface.
    vol_day = pd.Timestamp(datetime.datetime.strptime(str(vol_date), '%Y%m%d'))

    logger.info('')
    logger.info("[%d/%d] Start %d'th iteration." % (idate+1, len(vol_dates), idate+1))
    logger.info('Considering %s index. Creating vol surface at date: %s.' % (
        config.DATA_TYPE, vol_day.strftime('%Y/%m/%d')))

    try:
        assert(vol_day in all_days_init)
    except Exception as e:
        logger.fatal('No param data exists for date %s.' % vol_day.strftime('%Y/%m/%d'))
        raise e
    day_index = all_days_init.index(vol_day)
    try:
        assert(day_index > 0)
    except Exception as e:
        logger.fatal('No param data exists for yesterday of date %s.' %
                     vol_day.strftime('%Y/%m/%d'))
        raise e
    vol_yesterday = all_days_init[day_index-1]
    try:
        assert(vol_yesterday in coef_days)
    except Exception as e:
        logger.fatal('No coef data exists for yesterday of date %s: %s.' %
                     (vol_day.strftime('%Y/%m/%d'), vol_yesterday.strftime('%Y/%m/%d')))
        raise e
    logger.info('Using raw parameters of date: %s and vol surface of date: %s.' %
                (vol_day.strftime('%Y/%m/%d'), vol_yesterday.strftime('%Y/%m/%d')))

    # 5. Create vol surface.
    try:
        assert(os.path.isdir(config.RESULT_BASE_DIR))
    except Exception as e:
        logger.fatal('Invalid result directory: "%s."' % config.RESULT_BASE_DIR)
        raise e

    pred_alpha_coefs, pred_rho_coefs, pred_nu_coefs, \
        daily_alpha_train_data, daily_rho_train_data, daily_nu_train_data = create_vol_surface_at_day(
        logger, config, vol_day, vol_yesterday, market_data, raw_param_data, coef_data,
        alpha_loss_model, alpha_candidate_model,
        nu_loss_model, nu_candidate_model,
        rho_loss_model, rho_candidate_model, save_base_name='', plot_result=False,
        plot_curve=False, plot_smile=False, plot_surface=False)
        #rho_loss_model, rho_candidate_model, save_base_name='', plot_result=True,
        #plot_curve=True, plot_smile=False, plot_surface=False)

    # 6. Save the results.
    logger.info('Finally writing the results to excel format.')
    pred_coef_data = daily_coefs_to_coef_data(vol_day, pred_alpha_coefs, pred_rho_coefs, pred_nu_coefs, pred_coef_data)

    pred_alpha_selection_data = daily_train_data_to_selection_data(daily_alpha_train_data, pred_alpha_selection_data)
    pred_rho_selection_data = daily_train_data_to_selection_data(daily_rho_train_data, pred_rho_selection_data)
    pred_nu_selection_data = daily_train_data_to_selection_data(daily_nu_train_data, pred_nu_selection_data)

    save_coef_data_from_config(config, pred_coef_data)
    save_selection_data_from_config(
        config, pred_alpha_selection_data, pred_rho_selection_data, pred_nu_selection_data)
    logger.info('')

    logger.info('[%d/%d] Total Elapsed time: %.5f seconds.' % (idate+1, len(vol_dates), time.time() - start_time))

    INFO - 
    INFO - [1/179] Start 1'th iteration.
    INFO - Considering Kospi200 index. Creating vol surface at date: 2019/01/01.
    INFO - Using raw parameters of date: 2019/01/01 and vol surface of date: 2018/12/31.
    INFO - 
    INFO - Now creating vol surface.
    INFO - 
    INFO - Computed Alpha Coefficients: 0.087871, 0.098587, 4.864453, 0.004535, 0.059823
    INFO - Elapsed time:     2 seconds.
    INFO - 


[2.77777778e-03 5.55555556e-03 8.33333333e-03 ... 2.99444444e+00
 2.99722222e+00 3.00000000e+00]
[0.18365893 0.18338428 0.18311322 ... 0.15726902 0.15726343 0.15725784]
0.0


    INFO - Computed Rho Coefficients: -0.407504, -0.198663, 3.447327
    INFO - Elapsed time:     4 seconds.
    INFO - 


[2.77777778e-03 5.55555556e-03 8.33333333e-03 ... 2.99444444e+00
 2.99722222e+00 3.00000000e+00]
[-0.60427298 -0.60239773 -0.60054035 ... -0.40751013 -0.40751006
 -0.40751   ]
0.0


    INFO - Computed Nu Coefficients: 0.028728, 0.714049, -0.408832, -0.150529
    INFO - Elapsed time:     6 seconds.
    INFO - 
    INFO - Vol surface is created.
    INFO - Elapsed time:     6 seconds.
    INFO - 
    INFO - Finally writing the results to excel format.
    INFO - 
    INFO - [1/179] Total Elapsed time: 6.73699 seconds.
    INFO - 
    INFO - [2/179] Start 2'th iteration.
    INFO - Considering Kospi200 index. Creating vol surface at date: 2019/01/02.
    INFO - Using raw parameters of date: 2019/01/02 and vol surface of date: 2019/01/01.
    INFO - 
    INFO - Now creating vol surface.
    INFO - 


...

    INFO - Computed Nu Coefficients: 0.074626, 0.711151, -0.435964, -0.152942
    INFO - Elapsed time:     6 seconds.
    INFO - 
    INFO - Vol surface is created.
    INFO - Elapsed time:     6 seconds.
    INFO - 
    INFO - Finally writing the results to excel format.


[2.77777778e-03 5.55555556e-03 8.33333333e-03 ... 2.99444444e+00
 2.99722222e+00 3.00000000e+00]
[9.3266038  6.91077793 5.80070949 ... 0.35350032 0.35326921 0.35303839]
0.0


    INFO - 
    INFO - [36/179] Total Elapsed time: 238.04340 seconds.
    INFO - 
    INFO - [37/179] Start 37'th iteration.
    INFO - Considering Kospi200 index. Creating vol surface at date: 2019/02/20.
    INFO - Using raw parameters of date: 2019/02/20 and vol surface of date: 2019/02/19.
    INFO - 
    INFO - Now creating vol surface.
    INFO - 
    INFO - Computed Alpha Coefficients: 0.708058, -0.016494, 1.255575, -0.011214, 0.019823
    INFO - Elapsed time:     2 seconds.
    INFO - 


[2.77777778e-03 5.55555556e-03 8.33333333e-03 ... 2.99444444e+00
 2.99722222e+00 3.00000000e+00]
[0.12925938 0.12932053 0.12938153 ... 0.15849209 0.15850813 0.15852416]
0.0


KeyboardInterrupt: 

In [8]:
save_coef_data_from_config(config, pred_coef_data)
save_selection_data_from_config(
    config, pred_alpha_selection_data, pred_rho_selection_data, pred_nu_selection_data)

In [24]:
import scipy.stats as stats
print(stats.zscore([1, 2, 3, 4, 5, 7, 8, 9, 10, 1e100]))

[-0.33333333 -0.33333333 -0.33333333 -0.33333333 -0.33333333 -0.33333333
 -0.33333333 -0.33333333 -0.33333333  3.        ]


In [15]:
display(pred_alpha_selection_data.reset_index().reindex(columns=['REALDAY', 'STATUS', 'TTM', 'VALUE', 'SELECTED']).set_index('REALDAY'))

,STATUS,TTM,VALUE,SELECTED
REALDAY,,,,
2019-01-01,YESTERDAY,30,0.177668,X
2019-01-01,YESTERDAY,61,0.172778,O
2019-01-01,YESTERDAY,91,0.169451,X
2019-01-01,YESTERDAY,182,0.164450,O
2019-01-01,YESTERDAY,274,0.162550,X
...,...,...,...,...
2019-01-10,RAW,119,0.148179,O
2019-01-10,RAW,154,0.147492,O
2019-01-10,RAW,244,0.147025,X
